In [1]:
# import tensorflow as tf
# from tensorflow.keras.utils import plot_model
# from tqdm import tqdm_notebook

In [1]:
import pandas as pd
import numpy as np
from datetime import date, datetime
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l1_l2

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
credit = pd.read_csv('./train.csv')

In [3]:
credit_test = pd.read_csv('./test.csv')

In [4]:
credit

,user_id,device_id,age,sex,browser,source,country,purchase_value,signup_time,purchase_time,class
0,6b5aecb444b26,7a543b4bf3647,33,0,Safari,Ads,United States,48,2018-05-27 12:02:29,2018-08-19 14:09:55,False
1,f00edc3db68f0,c35ab03e7ff0f,26,1,Chrome,Direct,European Union,50,2018-06-06 08:09:12,2018-09-09 01:26:43,False
2,6f144a20b2e41,31a274c4b1d58,28,0,Chrome,Direct,United States,31,2018-03-29 18:24:17,2018-06-13 04:56:21,False
3,21db12c470157,74e7d74dbd61d,50,0,FireFox,Ads,United States,31,2018-06-18 02:49:33,2018-06-29 23:31:45,False
4,af1753dfd703c,b5f31ea91ae62,27,1,IE,SEO,United States,16,2018-01-31 18:26:38,2018-02-13 16:36:51,False
...,...,...,...,...,...,...,...,...,...,...,...
108795,85b246a47a137,c3342bf47d0f4,48,1,FireFox,SEO,NaN,32,2018-03-12 21:29:01,2018-03-20 18:57:27,False
108796,6ebeb734cd7d1,8c5d2be633d4b,34,0,Chrome,SEO,Australia,17,2018-05-17 02:34:36,2018-07-04 10:48:28,False
108797,5146b9d8ff25b,932913a6799ca,34,0,Chrome,Ads,Russian Federation,29,2018-01-21 03:07:31,2018-02-13 20:07:15,False
108798,db063895a5687,c3ae23109de70,19,0,Chrome,SEO,United States,42,2018-02-16 22:19:47,2018-03-08 01:50:58,False


In [5]:
credit_test

,user_id,device_id,age,sex,browser,source,country,purchase_value,signup_time,purchase_time
0,8fdb79dfdab19,5b4f85edff32c,32,1,Safari,SEO,United States,32,2018-02-05 04:39:18,2018-03-16 07:59:53
1,ae01bb5893cf5,3019f1e63a34b,47,1,Chrome,Direct,United States,30,2018-03-30 02:46:05,2018-05-15 15:33:45
2,563df4fb75a3d,431b43e6f56c8,26,1,Chrome,SEO,Panama,49,2018-02-16 04:53:25,2018-05-08 08:25:02
3,b21070f161c56,edb32ffdd016e,33,0,Safari,SEO,Italy,31,2018-04-28 00:15:22,2018-08-19 12:47:15
4,ee9775da22b94,32bcf0c089ae3,35,0,Chrome,Direct,United States,15,2018-05-08 16:42:44,2018-05-20 12:30:28
...,...,...,...,...,...,...,...,...,...,...
27195,cd16362b71572,2515db9e4437e,29,1,IE,SEO,United States,48,2018-03-08 05:02:33,2018-05-05 23:04:43
27196,132fb9f609701,58511ac9e2ebb,42,0,Safari,Ads,France,12,2018-05-15 23:23:29,2018-05-19 06:35:14
27197,4c6d7c325963f,8964657427d3d,41,1,Chrome,Ads,NaN,26,2018-06-08 15:39:11,2018-06-13 17:24:44
27198,61daa6f70df29,df795a2402c3f,36,1,IE,Ads,Romania,17,2018-06-25 12:47:27,2018-08-27 19:17:54


In [6]:
credit_combined = [credit, credit_test]
credit = pd.concat(credit_combined)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [7]:
credit

,age,browser,class,country,device_id,purchase_time,purchase_value,sex,signup_time,source,user_id
0,33,Safari,False,United States,7a543b4bf3647,2018-08-19 14:09:55,48,0,2018-05-27 12:02:29,Ads,6b5aecb444b26
1,26,Chrome,False,European Union,c35ab03e7ff0f,2018-09-09 01:26:43,50,1,2018-06-06 08:09:12,Direct,f00edc3db68f0
2,28,Chrome,False,United States,31a274c4b1d58,2018-06-13 04:56:21,31,0,2018-03-29 18:24:17,Direct,6f144a20b2e41
3,50,FireFox,False,United States,74e7d74dbd61d,2018-06-29 23:31:45,31,0,2018-06-18 02:49:33,Ads,21db12c470157
4,27,IE,False,United States,b5f31ea91ae62,2018-02-13 16:36:51,16,1,2018-01-31 18:26:38,SEO,af1753dfd703c
...,...,...,...,...,...,...,...,...,...,...,...
27195,29,IE,NaN,United States,2515db9e4437e,2018-05-05 23:04:43,48,1,2018-03-08 05:02:33,SEO,cd16362b71572
27196,42,Safari,NaN,France,58511ac9e2ebb,2018-05-19 06:35:14,12,0,2018-05-15 23:23:29,Ads,132fb9f609701
27197,41,Chrome,NaN,NaN,8964657427d3d,2018-06-13 17:24:44,26,1,2018-06-08 15:39:11,Ads,4c6d7c325963f
27198,36,IE,NaN,Romania,df795a2402c3f,2018-08-27 19:17:54,17,1,2018-06-25 12:47:27,Ads,61daa6f70df29


In [8]:
credit.isnull().sum()

age                   0
browser               0
class             27200
country           19766
device_id             0
purchase_time         0
purchase_value        0
sex                   0
signup_time           0
source                0
user_id               0
dtype: int64

In [9]:
# pd.set_option('display.max_rows', None)

In [10]:
# pd.reset_option("display.max_rows")

In [11]:
credit.dtypes

age                int64
browser           object
class             object
country           object
device_id         object
purchase_time     object
purchase_value     int64
sex                int64
signup_time       object
source            object
user_id           object
dtype: object

# 檢查欄位

### device_id
    觀察device_id重複出現時，若出現n筆，則可能有n-1筆class為True。
    預計處理方式：
        1. 依重複筆數分類 或
        2. 依重複筆數True的機率取代
    目前先以筆數分類。

In [12]:
credit['device_id'].unique()

array(['7a543b4bf3647', 'c35ab03e7ff0f', '31a274c4b1d58', ...,
       '58511ac9e2ebb', '8964657427d3d', 'df795a2402c3f'], dtype=object)

In [13]:
device_id = credit.groupby(['device_id'])[['user_id']].count()
device_id[device_id['user_id']>3].sort_values(['user_id'], ascending=False)

,user_id
device_id,
17e5f3b786435,20
f23d0361885d0,20
8d57801d3e4f7,19
d6c6850c7fa15,18
056d5de182b61,18
...,...
172c356af39b1,4
ac463333a355d,4
5dda911a9cd6b,4


In [14]:
credit[credit['device_id']=='d6c6850c7fa15']

,age,browser,class,country,device_id,purchase_time,purchase_value,sex,signup_time,source,user_id
1882,45,Chrome,True,China,d6c6850c7fa15,2018-01-12 17:05:12,33,1,2018-01-12 17:58:05,Ads,52bb3b81202e5
6023,45,Chrome,True,China,d6c6850c7fa15,2018-01-12 17:58:46,33,1,2018-01-12 17:59:41,Ads,5c33f21b0d7db
17727,45,Chrome,True,China,d6c6850c7fa15,2018-01-12 17:27:37,33,1,2018-01-12 17:19:59,Ads,49f029c92a2f0
21721,45,Chrome,True,China,d6c6850c7fa15,2018-01-12 17:37:00,33,1,2018-01-12 17:23:16,Ads,6d6dba8199480
24183,45,Chrome,True,China,d6c6850c7fa15,2018-01-12 17:26:08,33,1,2018-01-12 17:45:36,Ads,ad48e352fb3ff
30945,45,Chrome,True,China,d6c6850c7fa15,2018-01-12 17:39:23,33,1,2018-01-12 17:06:05,Ads,6939aaa4624a0
31499,45,Chrome,True,China,d6c6850c7fa15,2018-01-12 17:11:28,33,1,2018-01-12 17:49:21,Ads,e224612a0c62c
33075,45,Chrome,True,China,d6c6850c7fa15,2018-01-12 17:18:56,33,1,2018-01-12 17:00:53,Ads,5c0b1836fed44
36539,45,Chrome,True,China,d6c6850c7fa15,2018-01-12 17:44:47,33,1,2018-01-12 17:21:24,Ads,0f3097ab3a357
48316,45,Chrome,True,China,d6c6850c7fa15,2018-01-12 17:53:11,33,1,2018-01-12 17:18:13,Ads,a4042f866e2ed


In [15]:
credit[credit['device_id']=='7e53d1e7c035f']

,age,browser,class,country,device_id,purchase_time,purchase_value,sex,signup_time,source,user_id
6182,24,Safari,False,United States,7e53d1e7c035f,2018-03-29 07:20:27,65,0,2018-01-08 04:53:52,Ads,86a17f4814c25
62469,24,Safari,True,United States,7e53d1e7c035f,2018-01-08 04:36:17,65,0,2018-01-08 04:50:34,Ads,973500e979842
64937,24,Safari,True,United States,7e53d1e7c035f,2018-01-08 04:59:11,65,0,2018-01-08 04:06:44,Ads,28aebc4b03e61
88064,24,Safari,True,United States,7e53d1e7c035f,2018-01-08 04:29:15,65,0,2018-01-08 04:28:30,Ads,0d7a1989aa276
8934,24,Safari,NaN,United States,7e53d1e7c035f,2018-01-08 04:05:59,65,0,2018-01-08 04:10:51,Ads,6f864c25e20c1
20290,24,Safari,NaN,United States,7e53d1e7c035f,2018-01-08 04:01:47,65,0,2018-01-08 04:18:59,Ads,ad24946f489ff


In [16]:
credit['device_id'] = credit.apply(lambda x: credit['device_id'].map(credit['device_id'].value_counts()))

In [17]:
credit

,age,browser,class,country,device_id,purchase_time,purchase_value,sex,signup_time,source,user_id
0,33,Safari,False,United States,1,2018-08-19 14:09:55,48,0,2018-05-27 12:02:29,Ads,6b5aecb444b26
1,26,Chrome,False,European Union,1,2018-09-09 01:26:43,50,1,2018-06-06 08:09:12,Direct,f00edc3db68f0
2,28,Chrome,False,United States,1,2018-06-13 04:56:21,31,0,2018-03-29 18:24:17,Direct,6f144a20b2e41
3,50,FireFox,False,United States,1,2018-06-29 23:31:45,31,0,2018-06-18 02:49:33,Ads,21db12c470157
4,27,IE,False,United States,1,2018-02-13 16:36:51,16,1,2018-01-31 18:26:38,SEO,af1753dfd703c
...,...,...,...,...,...,...,...,...,...,...,...
27195,29,IE,NaN,United States,1,2018-05-05 23:04:43,48,1,2018-03-08 05:02:33,SEO,cd16362b71572
27196,42,Safari,NaN,France,1,2018-05-19 06:35:14,12,0,2018-05-15 23:23:29,Ads,132fb9f609701
27197,41,Chrome,NaN,NaN,1,2018-06-13 17:24:44,26,1,2018-06-08 15:39:11,Ads,4c6d7c325963f
27198,36,IE,NaN,Romania,1,2018-08-27 19:17:54,17,1,2018-06-25 12:47:27,Ads,61daa6f70df29


### age
    觀察age於65歲以上True出現次數較少，其餘年齡層目測約10%。
    預計處理方式：
        1. 切25以下、25~65、65以上。
        2. 每五歲or十歲分層。
    目前先以每十歲分層。

In [18]:
credit['age'].unique()

array([33, 26, 28, 50, 27, 24, 38, 35, 40, 30, 32, 41, 52, 36, 23, 39, 48,
       46, 20, 34, 43, 37, 29, 22, 31, 57, 42, 18, 44, 45, 21, 19, 47, 51,
       25, 53, 49, 54, 61, 55, 56, 59, 63, 66, 65, 62, 64, 58, 67, 60, 68,
       69, 72, 70, 71, 76, 73, 74])

In [19]:
credit[(credit['age'] >= 50) & (credit['age'] <75)].groupby(['class'])[['user_id']].count()

,user_id
class,
False,3821
True,436


In [20]:
credit[(credit['age'] <= 25)].groupby(['class'])[['user_id']].count()

,user_id
class,
False,20659
True,2051


In [21]:
credit[(credit['age'] >= 65)].groupby(['class'])[['user_id']].count()

,user_id
class,
False,45
True,3


In [22]:
credit['age'] = np.where(credit['age'] <= 25, 0, credit['age'])
credit['age'] = np.where((credit['age'] > 25) & (credit['age'] <= 35), 1, credit['age'])
credit['age'] = np.where((credit['age'] > 35) & (credit['age'] <= 45), 2, credit['age'])
credit['age'] = np.where((credit['age'] > 45) & (credit['age'] <= 55), 3, credit['age'])
credit['age'] = np.where((credit['age'] > 55) & (credit['age'] <= 65), 4, credit['age'])
credit['age'] = np.where(credit['age'] > 65, 5, credit['age'])

In [23]:
credit

,age,browser,class,country,device_id,purchase_time,purchase_value,sex,signup_time,source,user_id
0,1,Safari,False,United States,1,2018-08-19 14:09:55,48,0,2018-05-27 12:02:29,Ads,6b5aecb444b26
1,1,Chrome,False,European Union,1,2018-09-09 01:26:43,50,1,2018-06-06 08:09:12,Direct,f00edc3db68f0
2,1,Chrome,False,United States,1,2018-06-13 04:56:21,31,0,2018-03-29 18:24:17,Direct,6f144a20b2e41
3,3,FireFox,False,United States,1,2018-06-29 23:31:45,31,0,2018-06-18 02:49:33,Ads,21db12c470157
4,1,IE,False,United States,1,2018-02-13 16:36:51,16,1,2018-01-31 18:26:38,SEO,af1753dfd703c
...,...,...,...,...,...,...,...,...,...,...,...
27195,1,IE,NaN,United States,1,2018-05-05 23:04:43,48,1,2018-03-08 05:02:33,SEO,cd16362b71572
27196,2,Safari,NaN,France,1,2018-05-19 06:35:14,12,0,2018-05-15 23:23:29,Ads,132fb9f609701
27197,2,Chrome,NaN,NaN,1,2018-06-13 17:24:44,26,1,2018-06-08 15:39:11,Ads,4c6d7c325963f
27198,2,IE,NaN,Romania,1,2018-08-27 19:17:54,17,1,2018-06-25 12:47:27,Ads,61daa6f70df29


### sex, browser, source, country
    判斷上述欄位與數值較無關係。
    預計處理方式：
        1. country有空值，先補unknown。
        2. one-hot encoding。

In [24]:
credit['country'].fillna('Unknown', inplace = True) 

In [25]:
null = credit.groupby(['country'])[['user_id']].count()
null.sort_values(['user_id'], ascending=False)

,user_id
country,
United States,52206
Unknown,19766
China,10857
Japan,6577
United Kingdom,4028
...,...
Madagascar,1
Niger,1
Gambia,1


In [26]:
credit['sex'].unique()

array([0, 1])

In [27]:
credit['browser'].unique()

array(['Safari', 'Chrome', 'FireFox', 'IE', 'Opera'], dtype=object)

In [28]:
credit['source'].unique()

array(['Ads', 'Direct', 'SEO'], dtype=object)

In [29]:
credit['country'].unique()

array(['United States', 'European Union', 'Unknown', 'Germany', 'Japan',
       'Hong Kong', 'Netherlands', 'France', 'Norway', 'China', 'Poland',
       'Spain', 'Italy', 'United Kingdom', 'Korea Republic of', 'Brazil',
       'Russian Federation', 'Pakistan', 'Kenya', 'Denmark', 'Peru',
       'Australia', 'Switzerland', 'Belgium', 'Bulgaria', 'Viet Nam',
       'Singapore', 'Czech Republic', 'Finland', 'India', 'Sweden',
       'Antigua and Barbuda', 'Turkey', 'Mexico', 'Latvia', 'Colombia',
       'Canada', 'Argentina', 'South Africa', 'Lebanon',
       'Moldova Republic of', 'Israel', 'Iran (ISLAMIC Republic Of)',
       'Taiwan; Republic of China (ROC)', 'Hungary',
       'Syrian Arab Republic', 'Costa Rica', 'Greece', 'Iraq', 'Ecuador',
       'Croatia (LOCAL Name: Hrvatska)', 'Georgia', 'Thailand', 'Ukraine',
       'Austria', 'New Zealand', 'Chile', 'Libyan Arab Jamahiriya',
       'Ireland', 'Indonesia', 'Tunisia', 'Saudi Arabia', 'Portugal',
       'Romania', 'El Salvador', 

### purchase_value
    看不出特別關係。
    預計處理方式：reshape(0, 1)

In [30]:
credit['purchase_value'].unique()

array([ 48,  50,  31,  16,  86,  28,  21,  52,  14,  13,  47,  36,  22,
        49,  39,  66,  43,  69,  26,  41,  94,  27,  63,   9,  24,  65,
        59,  46,  34,  29,  38,  15,  19,  61,  20,  25,  33,  45,  35,
        17,  54,  40,  12,  56,  23,  32,  30,  57,  67,  37,  60,  77,
        55,  62,  76,  10,  11,  18,  64,  53,  42,  87,  83,  72,  78,
        85,  58,  75,  70,  44,  51,  73,  90,  81,  68,  71,  79,  93,
        74,  80,  82, 104,  88,  92,  95,  84, 102,  89, 128,  96, 112,
       107,  91, 101, 108,  98, 106, 100,  99, 110, 111, 115, 103, 119,
       109, 113, 105, 116,  97, 127, 122, 117, 121, 129, 114, 123, 118,
       125, 154, 132, 120, 140])

In [31]:
purchase_value = credit.groupby(['purchase_value'])[['user_id']].count()
purchase_value[purchase_value['user_id']>3].sort_values(['user_id'], ascending=False)

,user_id
purchase_value,
28,2995
32,2887
27,2872
24,2830
26,2825
...,...
109,9
111,8
113,7


In [32]:
credit[(credit['purchase_value'] == 28)].groupby(['class'])[['user_id']].count()

,user_id
class,
False,2168
True,242


In [33]:
scaler = MinMaxScaler()
credit['purchase_value'] = scaler.fit_transform(credit['purchase_value'].values.reshape(-1, 1))

### signup_time, purchase_time
    觀察device_id重複出現時，若出現n筆，則可能有n-1筆class為True。
    預計處理方式：
        1. 依重複筆數分類 或
        2. 依重複筆數True的機率取代
    目前先以筆數分類。

In [34]:
credit['signup_time'] =  pd.to_datetime(credit['signup_time'], format='%Y-%m-%d %H:%M:%S.%f')
credit['purchase_time'] =  pd.to_datetime(credit['purchase_time'], format='%Y-%m-%d %H:%M:%S.%f')

In [35]:
credit.dtypes

age                        int64
browser                   object
class                     object
country                   object
device_id                  int64
purchase_time     datetime64[ns]
purchase_value           float64
sex                        int64
signup_time       datetime64[ns]
source                    object
user_id                   object
dtype: object

In [36]:
credit

,age,browser,class,country,device_id,purchase_time,purchase_value,sex,signup_time,source,user_id
0,1,Safari,False,United States,1,2018-08-19 14:09:55,0.268966,0,2018-05-27 12:02:29,Ads,6b5aecb444b26
1,1,Chrome,False,European Union,1,2018-09-09 01:26:43,0.282759,1,2018-06-06 08:09:12,Direct,f00edc3db68f0
2,1,Chrome,False,United States,1,2018-06-13 04:56:21,0.151724,0,2018-03-29 18:24:17,Direct,6f144a20b2e41
3,3,FireFox,False,United States,1,2018-06-29 23:31:45,0.151724,0,2018-06-18 02:49:33,Ads,21db12c470157
4,1,IE,False,United States,1,2018-02-13 16:36:51,0.048276,1,2018-01-31 18:26:38,SEO,af1753dfd703c
...,...,...,...,...,...,...,...,...,...,...,...
27195,1,IE,NaN,United States,1,2018-05-05 23:04:43,0.268966,1,2018-03-08 05:02:33,SEO,cd16362b71572
27196,2,Safari,NaN,France,1,2018-05-19 06:35:14,0.020690,0,2018-05-15 23:23:29,Ads,132fb9f609701
27197,2,Chrome,NaN,Unknown,1,2018-06-13 17:24:44,0.117241,1,2018-06-08 15:39:11,Ads,4c6d7c325963f
27198,2,IE,NaN,Romania,1,2018-08-27 19:17:54,0.055172,1,2018-06-25 12:47:27,Ads,61daa6f70df29


In [37]:
#檢查weekday-class樣態
# credit.groupby(['signup_weekday'])[['user_id']].count()
# credit[credit['class'] == True].groupby(['signup_weekday'])[['user_id']].count()
# credit.groupby(['purchase_weekday'])[['user_id']].count()
# credit[credit['class'] == True].groupby(['purchase_weekday'])[['user_id']].count()

In [38]:
credit['signup_weekday'] = credit['signup_time'].dt.day_name()
credit['purchase_weekday'] = credit['purchase_time'].dt.day_name()

credit['diff_time'] = credit['purchase_time'] - credit['signup_time']

credit['signup_date'] = pd.to_datetime(credit['signup_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.date
credit['purchase_date'] = pd.to_datetime(credit['purchase_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.date

credit['signup_year'] = pd.to_datetime(credit['signup_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.year
credit['purchase_year'] = pd.to_datetime(credit['purchase_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.year
credit['signup_month'] = pd.to_datetime(credit['signup_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.month
credit['purchase_month'] = pd.to_datetime(credit['purchase_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.month
credit['signup_day'] = pd.to_datetime(credit['signup_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.day
credit['purchase_day'] = pd.to_datetime(credit['purchase_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.day

credit['signup_time'] = pd.to_datetime(credit['signup_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.hour+pd.to_datetime(credit['signup_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.minute/60+pd.to_datetime(credit['signup_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.second/3600
credit['purchase_time'] = pd.to_datetime(credit['purchase_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.hour+pd.to_datetime(credit['purchase_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.minute/60+pd.to_datetime(credit['purchase_time'], format='%Y-%m-%d %H:%M:%S.%f').dt.second/3600
credit['signup_time_sin'] = np.sin(2 * np.pi * credit['signup_time']/24)
credit['signup_time_cos'] = np.cos(2 * np.pi * credit['signup_time']/24)
credit['purchase_time_sin'] = np.sin(2 * np.pi * credit['purchase_time']/24)
credit['purchase_time_cos'] = np.cos(2 * np.pi * credit['purchase_time']/24)

In [39]:
credit['diff_time'] = scaler.fit_transform(credit['diff_time'].values.reshape(-1, 1))
#為什麼轉完沒有負數？？？

In [40]:
credit

,age,browser,class,country,device_id,purchase_time,purchase_value,sex,signup_time,source,...,signup_year,purchase_year,signup_month,purchase_month,signup_day,purchase_day,signup_time_sin,signup_time_cos,purchase_time_sin,purchase_time_cos
0,1,Safari,False,United States,1,14.165278,0.268966,0,12.041389,Ads,...,2018,2018,5,8,27,19,-0.010835,-0.999941,-0.536993,-0.843587
1,1,Chrome,False,European Union,1,1.445278,0.282759,1,8.153333,Direct,...,2018,2018,6,9,6,9,0.845262,-0.534352,0.369409,0.929267
2,1,Chrome,False,United States,1,4.939167,0.151724,0,18.404722,Direct,...,2018,2018,3,6,29,13,-0.994392,0.105758,0.961682,0.274169
3,3,FireFox,False,United States,1,23.529167,0.151724,0,2.825833,Ads,...,2018,2018,6,6,18,29,0.674141,0.738602,-0.122952,0.992413
4,1,IE,False,United States,1,16.614167,0.048276,1,18.443889,SEO,...,2018,2018,1,2,31,13,-0.993255,0.115948,-0.934903,-0.354903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27195,1,IE,NaN,United States,1,23.078611,0.268966,1,5.042500,SEO,...,2018,2018,3,5,8,5,0.968746,0.248056,-0.238887,0.971047
27196,2,Safari,NaN,France,1,6.587222,0.020690,0,23.391389,Ads,...,2018,2018,5,5,15,19,-0.158661,0.987333,0.988206,-0.153130
27197,2,Chrome,NaN,Unknown,1,17.412222,0.117241,1,15.653056,Ads,...,2018,2018,6,6,8,13,-0.817103,-0.576492,-0.988184,-0.153273
27198,2,IE,NaN,Romania,1,19.298333,0.055172,1,12.790833,Ads,...,2018,2018,6,8,25,27,-0.205564,-0.978644,-0.942787,0.333396


In [41]:
y = to_categorical(credit['class'][:108800])
y[0:10]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [42]:
credit = credit.drop(['user_id', 'class', 'signup_time', 'purchase_time', 'signup_date', 'purchase_date'], axis=1)

In [43]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136000 entries, 0 to 27199
Data columns (total 20 columns):
age                  136000 non-null int64
browser              136000 non-null object
country              136000 non-null object
device_id            136000 non-null int64
purchase_value       136000 non-null float64
sex                  136000 non-null int64
source               136000 non-null object
signup_weekday       136000 non-null object
purchase_weekday     136000 non-null object
diff_time            136000 non-null float64
signup_year          136000 non-null int64
purchase_year        136000 non-null int64
signup_month         136000 non-null int64
purchase_month       136000 non-null int64
signup_day           136000 non-null int64
purchase_day         136000 non-null int64
signup_time_sin      136000 non-null float64
signup_time_cos      136000 non-null float64
purchase_time_sin    136000 non-null float64
purchase_time_cos    136000 non-null float64
dtypes: float

In [44]:
credit.describe()

,age,device_id,purchase_value,sex,diff_time,signup_year,purchase_year,signup_month,purchase_month,signup_day,purchase_day,signup_time_sin,signup_time_cos,purchase_time_sin,purchase_time_cos
count,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.0,136000.0,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000
mean,1.262963,1.612603,0.192828,0.584169,0.475878,2018.0,2018.0,4.148382,6.010382,14.735610,15.271132,-0.003177,-0.001512,-0.002116,-0.002562
std,0.907232,2.358204,0.126449,0.492866,0.301364,0.0,0.0,2.257551,2.659645,8.731475,8.817054,0.708511,0.705696,0.707272,0.706939
min,0.000000,1.000000,0.000000,0.000000,0.000000,2018.0,2018.0,1.000000,1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,1.000000,1.000000,0.089655,0.000000,0.211465,2018.0,2018.0,2.000000,4.000000,7.000000,8.000000,-0.713569,-0.705098,-0.709571,-0.708802
50%,1.000000,1.000000,0.179310,1.000000,0.475392,2018.0,2018.0,4.000000,6.000000,14.000000,15.000000,-0.002254,-0.004291,-0.006181,-0.009454
75%,2.000000,1.000000,0.275862,1.000000,0.737314,2018.0,2018.0,6.000000,8.000000,22.000000,23.000000,0.705936,0.703756,0.705923,0.704325
max,5.000000,20.000000,1.000000,1.000000,1.000000,2018.0,2018.0,8.000000,12.000000,31.000000,31.000000,1.000000,1.000000,1.000000,1.000000


In [45]:
data_dum = pd.get_dummies(credit)
credit = pd.DataFrame(data_dum)

In [46]:
credit_test = credit[108800:]

In [47]:
x=credit[:108800]
x.shape, y.shape

((108800, 218), (108800, 2))

In [48]:
random_seed = 92023  # set seed for same train test data split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=random_seed)

In [49]:
print("shape of X_train: ", x_train.shape)

shape of X_train:  (87040, 218)


In [50]:
print("shape of X_test: ", x_test.shape)

shape of X_test:  (21760, 218)


In [51]:
#clf = RandomForestClassifier(n_estimators=20)
clf = RandomForestClassifier(n_estimators=60, criterion="gini", max_depth=None, bootstrap=True, random_state=None)

In [52]:
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=60,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [53]:
y_pred = clf.predict(x_test)

In [54]:
accuracy_score(y_test, y_pred)

0.9574448529411764

In [55]:
clf.feature_importances_

array([1.48343173e-02, 1.52897617e-01, 3.25656523e-02, 7.12284606e-03,
       2.10892643e-01, 0.00000000e+00, 0.00000000e+00, 4.26561640e-02,
       1.50414487e-01, 3.91946237e-02, 4.80398553e-02, 3.94209841e-02,
       3.90119365e-02, 3.86867879e-02, 4.08904558e-02, 5.58410753e-03,
       4.14964082e-03, 5.06438205e-03, 1.36218514e-03, 4.19665890e-03,
       3.75290941e-08, 1.63433674e-06, 3.16682116e-04, 2.30092442e-06,
       5.66186840e-08, 5.73226969e-04, 6.27685297e-05, 9.34965156e-04,
       3.50514656e-04, 5.20775004e-05, 2.44028746e-06, 3.78061871e-07,
       6.84956102e-05, 4.70049727e-08, 5.83404547e-05, 4.57201925e-04,
       4.60355949e-08, 2.83090081e-09, 2.64799738e-09, 1.28601737e-06,
       1.49579281e-04, 0.00000000e+00, 6.34425674e-05, 3.49510567e-07,
       1.33305780e-03, 0.00000000e+00, 2.52784690e-06, 6.53775920e-05,
       1.96158557e-08, 1.05468682e-06, 4.91421036e-08, 1.42140286e-03,
       3.02804123e-07, 3.27017864e-08, 4.45231225e-04, 3.19809059e-03,
      

In [56]:
y_pred = clf.predict(credit_test)
print(y_pred[:10,:])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [57]:
y_pred

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [59]:
y_label = np.argmax(y_pred, axis=1) # 取機率值最高的類別當作預測結果
print(y_label[:10])

[0 0 0 0 0 0 0 0 1 0]


In [60]:
test = pd.read_csv('./test.csv')

In [62]:
submission = pd.DataFrame()
submission['user_id'] = test.user_id
submission['class'] = y_label
submission

,user_id,class
0,8fdb79dfdab19,0
1,ae01bb5893cf5,0
2,563df4fb75a3d,0
3,b21070f161c56,0
4,ee9775da22b94,0
...,...,...
27195,cd16362b71572,0
27196,132fb9f609701,0
27197,4c6d7c325963f,0
27198,61daa6f70df29,0


In [63]:
submission.to_csv('submission.csv',index=False)